In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

# Sequantial Runnable

In [6]:
from langchain_core.runnables import RunnableSequence

llm = ChatOllama(model="llama3.2:latest")
template = PromptTemplate(
    template = "Give me a joke about {topic}",
    input_variables = ["topic"])
parser = StrOutputParser()

runnable = RunnableSequence(template, llm, parser)
print(runnable.invoke({"topic": "programming"}))  # "Why did the programmer quit his job?\n\nBecause he didn't get arrays! (get it?)"runnable.invoke({"topic": "programming"})

Here's one:

Why did the programmer quit his job?

Because he didn't get arrays of opportunities! (get it?)


# Runnable Parallel

In [7]:
from langchain_core.runnables import RunnableParallel

prompt1 = PromptTemplate(
    template = "Give me a tweet on {topic}",
    input_variables = ["topic"])

prompt2 = PromptTemplate(
    template = "Give me a linkedin post on {topic}",
    input_variables = ["topic"])

llm = ChatOllama(model="llama3.2:latest")
parser = StrOutputParser()

#Note that Runnable Parallel takes a dictionary where the keys are the names of the runnables and the values are the runnables themselves. When we invoke the RunnableParallel, it will return a dictionary where the keys are the same as the input dictionary and the values are the outputs of the respective runnables.
runnable_parallel = RunnableParallel({
    "tweet": RunnableSequence(prompt1, llm, parser),
    "linkedin_post": RunnableSequence(prompt2, llm, parser)
})

print(runnable_parallel.invoke({"topic": "AI"}))

{'tweet': '"AI is not just a tool, it\'s a reflection of our future. As we continue to innovate and improve, let\'s remember that with great power comes great responsibility. How do you think AI will shape our world in the next 5 years? #AI #FutureOfTech"', 'linkedin_post': 'Here\'s a potential LinkedIn post about AI:\n\n**"The Future of Work: Embracing the Power of Artificial Intelligence**\n\nAs we continue to navigate the ever-changing landscape of work, one thing is clear: Artificial Intelligence (AI) is here to stay. From automation to personalization, AI is transforming industries and revolutionizing the way we work.\n\nBut what does this mean for you? How can you harness the power of AI to enhance your skills, boost productivity, and drive innovation?\n\nHere are a few ways to get started:\n\n **Upskill and reskill**: Invest in training and development programs that focus on emerging technologies like AI, data science, and machine learning.\n\n **Leverage AI tools**: Explore AI-

# Runnable PassThrough

### Runnable Passthrough just takes the input and prints the same output

Walkthrought: prompt1 --> model --> generate joke then 2 parallel chains, one for printing joke (Runnable Passthrough) other for explaining the joke (prompt2 --> model --> explain joke)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

llm = ChatOllama(model="llama3.2:latest")

prompt1 = PromptTemplate(
    template = "Give me a joke on {topic}",
    input_variables = ["topic"])

prompt2 = PromptTemplate(
    template = "Explain me this joke: {joke}",
    input_variables = ["joke"])

parser = StrOutputParser()

joke_generator = RunnableSequence(prompt1, llm, parser)

parallel_chain = RunnableParallel({
    "joke": RunnablePassthrough(),
    "explanation": RunnableSequence(prompt2, llm, parser)
})

runnable = RunnableSequence(joke_generator, parallel_chain)
print(runnable.invoke({"topic": "AI"}))


{'joke': 'Why did the AI program go to therapy?\n\nBecause it was struggling with its "bytes" of emotions! (get it?)', 'explanation': 'A clever play on words!\n\nThe joke relies on a pun, which is a type of wordplay that exploits multiple meanings or sounds of words.\n\nIn this case, the punchline "bytes of emotions" has a double meaning:\n\n1. In computing, a "byte" refers to a unit of digital information.\n2. Emotions are feelings or emotional states.\n\nSo, when the joke says the AI program is struggling with its "bytes" of emotions, it\'s playing on this dual meaning. It\'s as if the AI is having trouble managing its own digital code (bytes) and also dealing with emotional turmoil. The humor comes from the unexpected twist on the usual meaning of "bytes"!\n\nThe use of parentheses "(get it?)" also implies that the speaker expects the listener to catch the pun and appreciate the clever wordplay.\n\nI hope this explanation helped clarify the joke!'}


# Runnable Lambda

It is used to make functions into a runnable

Walkthrough: prompt--> model --> joke. Then we have a parallel runnable, first one just prints the joke (RunnablePassthrough) and the second one is to count no.of words in the joke (RunnableLambda)

In [11]:
from langchain_core.runnables import RunnableLambda

def no_of_words(text):
    return len(text.split())

prompt = PromptTemplate(
    template = "Give me a joke on {topic}",
    input_variables = ["topic"])
llm = ChatOllama(model="llama3.2:latest")
parser = StrOutputParser()

joke_generator = RunnableSequence(prompt, llm, parser)  

parallel_chain = RunnableParallel({
    "joke": RunnablePassthrough(),
    "no_of_words": RunnableLambda(no_of_words)})

runnable = RunnableSequence(joke_generator, parallel_chain)
result = runnable.invoke({"topic": "AI"})
print(result)

{'joke': 'Why did the AI program go to therapy?\n\nBecause it was struggling to process its emotions.', 'no_of_words': 16}


# Runnable Branch

It is basically used for conditional logics. 

Walkthrough: Prompt --> llm --> response. Then if len(response) is greater than 300, we summarise is or else we print it as it is.


In [15]:
from langchain_core.runnables import RunnableBranch

prompt = PromptTemplate(
    template = "Give me detailed report on {topic}",
    input_variables = ["topic"])

prompt2 = PromptTemplate(
    template = "Give me a short summary of {text}",
    input_variables = ["text"])

llm = ChatOllama(model="llama3.2:latest")
parser = StrOutputParser()  

report_generator = RunnableSequence(prompt, llm, parser)

branch = RunnableBranch(
    ((lambda x: len(x.split()) > 300), RunnableSequence(prompt2, llm, parser)),
    RunnablePassthrough()
)

runnable = RunnableSequence(report_generator, branch)
result = runnable.invoke({"topic": "AI"})
print(result)

This report provides a thorough and well-structured overview of Artificial Intelligence (AI), covering its history, types, applications, machine learning, deep learning, ethics, challenges, and future prospects.

The report begins by introducing AI as the development of computer systems that can perform tasks typically requiring human intelligence, such as learning, problem-solving, and decision-making. It then delves into the history of AI, tracing back to ancient Greece and highlighting the contributions of pioneers like John McCarthy who coined the term "Artificial Intelligence" in 1956.

The report categorizes AI into three types: Narrow or Weak AI, General or Strong AI, and Superintelligence. Narrow AI is designed to perform specific tasks, while general AI aims to create machines that can think and learn like humans. Superintelligence refers to an intelligent machine that surpasses human intelligence in all domains.

AI has numerous applications across various industries, includi